In [1]:
from typing import Any

import numpy
import polars
from datasets import Dataset, load_dataset
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

In [2]:
dataset = load_dataset(
	"csv", data_dir="../data", data_files={"train": "train.csv", "test": "test.csv"}, keep_in_memory=True, num_proc=8
)

In [3]:
dataset_split = dataset["train"].train_test_split(test_size=0.1, keep_in_memory=True)
dataset["train"], dataset["validation"] = dataset_split["train"], dataset_split["test"]

In [4]:
dataset = dataset.map(
	lambda samples: {
		"play years_0": [1 if sample == 0 else 0 for sample in samples],
		"play years_1": [1 if sample == 1 else 0 for sample in samples],
		"play years_2": [1 if sample == 2 else 0 for sample in samples],
	},
	input_columns=["play years"],
	remove_columns=["play years"],
	batched=True,
	num_proc=8,
)

Map (num_proc=8):   0%|          | 0/23868 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/7488 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/2652 [00:00<?, ? examples/s]

In [5]:
dataset = dataset.map(
	lambda samples: {
		"level_0": [1 if sample == 0 else 0 for sample in samples],
		"level_1": [1 if sample == 1 else 0 for sample in samples],
		"level_2": [1 if sample == 2 else 0 for sample in samples],
	},
	input_columns=["level"],
	remove_columns=["level"],
	batched=True,
	num_proc=8,
)

Map (num_proc=8):   0%|          | 0/23868 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/7488 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/2652 [00:00<?, ? examples/s]

In [6]:
index_columns: list = ["data_ID", "player_ID"]
label_columns: list = [
	"gender",
	"hold racket handed",
	"play years_0",
	"play years_1",
	"play years_2",
	"level_0",
	"level_1",
	"level_2",
]
data_columns: list = list(set(dataset["train"].column_names).difference(set(index_columns + label_columns)))

In [7]:
train_dataset: polars.DataFrame = dataset["train"].to_polars()
train_data: numpy.ndarray = train_dataset[data_columns].to_numpy()
train_label: dict[str, numpy.ndarray] = {label: train_dataset[label].to_numpy() for label in label_columns}

In [8]:
validation_dataset: polars.DataFrame = dataset["validation"].to_polars()
validation_data: numpy.ndarray = validation_dataset[data_columns].to_numpy()
validation_label: dict[str, numpy.ndarray] = {label: validation_dataset[label].to_numpy() for label in label_columns}

In [9]:
hyperparameter_grid: dict[str, list[Any]] = {
	"n_estimators": [50, 100, 200, 300],
	"max_depth": [None, 10, 20, 30, 40],
	"min_samples_split": [2, 5, 10, 15],
	"min_samples_leaf": [1, 2, 4, 6],
	"max_features": ["sqrt", "log2"],
	"bootstrap": [True, False],
}

In [10]:
best_classifiers: dict = {}
for label in label_columns:
	print(f"Tune classifier for {label}")
	classifier = RandomForestClassifier(random_state=37)

	tuner = GridSearchCV(classifier, hyperparameter_grid, cv=5, scoring="roc_auc_ovr", n_jobs=-1, verbose=1)

	tuner.fit(train_data, train_label[label])

	best_classifiers[label] = tuner.best_estimator_

	y_pred_proba = best_classifiers[label].predict_proba(validation_data)[:, 1]
	auc_score = roc_auc_score(validation_label[label], y_pred_proba, multi_class="ovr")
	print(f"AUC score of {label} classifier: {auc_score:.4f}")

Tune classifier for gender
Fitting 5 folds for each of 1280 candidates, totalling 6400 fits



KeyboardInterrupt



In [39]:
test_data: polars.DataFrame = dataset["test"].select_columns(data_columns).to_polars().to_numpy()

In [41]:
predictions: dict = {"data_ID": dataset["test"]["data_ID"]}
for label in label_columns:
	predictions[label] = best_classifiers[label].predict(test_data).tolist()

In [42]:
test_result = Dataset.from_dict(predictions)

In [43]:
test_result

Dataset({
    features: ['data_ID', 'gender', 'hold racket handed', 'play years_0', 'play years_1', 'play years_2', 'level_0', 'level_1', 'level_2'],
    num_rows: 7488
})

In [44]:
test_result.to_csv("../dist/result_rf.csv", num_proc=8)

Creating CSV from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

164833